In [3]:
# Cellule d'installation des bibliothèques
!pip install tensorflow opencv-python-headless matplotlib numpy scikit-learn -q
!pip install mlflow -q

In [5]:
# Accéder au secret
from google.colab import userdata
import os

GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
GITHUB_USER = 'M-Carre'
GITHUB_REPO_NAME = 'OC-autonomous-vehicle-segmentation'
CLONE_DIR = f"/content/{GITHUB_REPO_NAME}"

# Construire l'URL de clonage
clone_url = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{GITHUB_REPO_NAME}.git"

# Exécuter la commande git clone
# os.system() est parfois plus propre pour ça que ! avec f-strings complexes
os.system(f"git clone {clone_url} {CLONE_DIR}")

# Vérifier
if os.path.exists(CLONE_DIR):
    %cd {CLONE_DIR}
    !ls
    print(f"Dépôt cloné avec succès dans {CLONE_DIR}")
else:
    print(f"Échec du clonage du dépôt dans {CLONE_DIR}")

/content/OC-autonomous-vehicle-segmentation
README.md
Dépôt cloné avec succès dans /content/OC-autonomous-vehicle-segmentation


# Script GitHub

In [23]:
# ==============================================================================
# SCRIPT DE SYNCHRONISATION (DRIVE -> REPO LOCAL) ET PUSH VERS GITHUB (V3 avec Debug LS)
# ==============================================================================
import os
import shutil
from google.colab import userdata

# --- Configuration des Chemins et Fichiers ---
PROJECT_BASE_ON_DRIVE = '/content/drive/MyDrive/Colab Notebooks/OC/P8'
GIT_REPO_PATH_IN_COLAB = '/content/OC-autonomous-vehicle-segmentation'
GIT_BRANCH_NAME = 'main'

# Assurez-vous que cette liste correspond aux dossiers sur Drive où vous avez mis les .gitkeep
ITEMS_TO_SYNC = [
    "notebooks",
    "src",
    "api",
    "webapp",
    "reports",
    "models",
    "scripts",    # Assurez-vous que ce dossier existe sur Drive avec un .gitkeep si vous le listez
    ".gitignore", # Si vous le gérez sur Drive
    "README.md"   # Si vous le gérez sur Drive
]
# ------------------------------------------------------------------------------

def sync_and_push_to_github_v3():
    print("### 1. VÉRIFICATION DES CHEMINS ###")
    if not os.path.exists(PROJECT_BASE_ON_DRIVE):
        print(f"ERREUR : Le dossier source sur Drive '{PROJECT_BASE_ON_DRIVE}' n'existe pas.")
        return
    if not os.path.exists(GIT_REPO_PATH_IN_COLAB):
        print(f"ERREUR : Le dossier du dépôt Git cloné '{GIT_REPO_PATH_IN_COLAB}' n'existe pas.")
        return
    print(f"Source du projet sur Drive : {PROJECT_BASE_ON_DRIVE}")
    print(f"Dépôt Git local dans Colab : {GIT_REPO_PATH_IN_COLAB}")

    print("\n### 2. NAVIGATION VERS LE RÉPERTOIRE DU DÉPÔT GIT LOCAL ###")
    try:
        os.chdir(GIT_REPO_PATH_IN_COLAB)
        print(f"Changé de répertoire vers : {os.getcwd()}")
    except FileNotFoundError:
        print(f"ERREUR CRITIQUE : Impossible de naviguer vers le répertoire du dépôt Git '{GIT_REPO_PATH_IN_COLAB}'.")
        return

    print("\n### 3. MISE À JOUR DEPUIS LE DÉPÔT DISTANT (GIT PULL) ###")
    # Configurez l'identité Git si ce n'est pas déjà fait dans la session
    # !git config --global user.email "VOTRE_EMAIL_GITHUB@example.com"
    # !git config --global user.name "VOTRE_NOM_UTILISATEUR_GITHUB"

    print(f"Tentative de 'git pull origin {GIT_BRANCH_NAME}' --rebase...")
    # Utilisation de --rebase pour maintenir un historique propre
    pull_result = os.system(f"git pull origin {GIT_BRANCH_NAME} --rebase")
    if pull_result != 0:
        print(f"AVERTISSEMENT : 'git pull --rebase' a rencontré un problème (code {pull_result}). Vérifiez les messages.")
        print("Si des conflits sont signalés, ils devront être résolus manuellement avant de continuer.")
        # Pour un script automatisé, il est difficile de gérer les conflits de rebase.
        # Une alternative serait de faire un 'git merge' si le rebase échoue,
        # ou simplement d'arrêter et de demander une intervention manuelle.
        # Pour l'instant, nous continuons, mais soyez conscient des problèmes potentiels.
    else:
        print("'git pull --rebase' terminé avec succès (ou pas de changements distants/conflits).")

    print("\n### 4. SYNCHRONISATION DES FICHIERS (DRIVE -> REPO LOCAL) ###")
    synced_item_count = 0
    for item_rel_path in ITEMS_TO_SYNC:
        source_item_abs_path = os.path.join(PROJECT_BASE_ON_DRIVE, item_rel_path)
        dest_item_abs_path = os.path.join(os.getcwd(), item_rel_path)

        if not os.path.exists(source_item_abs_path):
            # Si l'élément est listé dans ITEMS_TO_SYNC mais n'existe pas sur Drive, on l'ignore.
            # On pourrait aussi choisir de supprimer l'élément correspondant dans le repo local
            # si on veut une synchronisation stricte, mais c'est plus risqué.
            print(f"AVERTISSEMENT : L'élément source '{source_item_abs_path}' n'existe pas sur Drive. Ignoré pour la copie.")
            continue

        os.makedirs(os.path.dirname(dest_item_abs_path), exist_ok=True)

        if os.path.isdir(source_item_abs_path):
            if os.path.exists(dest_item_abs_path) and os.path.isdir(dest_item_abs_path): # S'assurer que c'est bien un dossier
                print(f"   Suppression du dossier de destination existant : {dest_item_abs_path}")
                shutil.rmtree(dest_item_abs_path)
            elif os.path.exists(dest_item_abs_path) and not os.path.isdir(dest_item_abs_path): # Si un fichier existe avec le même nom
                print(f"   Suppression du fichier de destination existant (au lieu d'un dossier) : {dest_item_abs_path}")
                os.remove(dest_item_abs_path)

            print(f"   Copie du dossier : '{source_item_abs_path}' vers '{dest_item_abs_path}'")
            shutil.copytree(source_item_abs_path, dest_item_abs_path)
            synced_item_count += 1
        elif os.path.isfile(source_item_abs_path):
            print(f"   Copie du fichier : '{source_item_abs_path}' vers '{dest_item_abs_path}'")
            shutil.copy2(source_item_abs_path, dest_item_abs_path)
            synced_item_count += 1

    if synced_item_count > 0: print(f"\n{synced_item_count} élément(s) listé(s) dans ITEMS_TO_SYNC et existant sur Drive ont été synchronisé(s).")
    else: print("\nAucun élément de ITEMS_TO_SYNC n'a été trouvé sur Drive pour la synchronisation ou la liste ITEMS_TO_SYNC est vide.")

    print("\n### 4b. INSPECTION DU CONTENU LOCAL APRÈS SYNCHRO ###")
    print(f"Contenu actuel de {os.getcwd()}:")
    !ls -A # -A pour montrer les fichiers cachés comme .gitkeep et .gitignore, mais pas . et ..

    # Liste des dossiers à inspecter (basée sur une structure typique)
    folders_to_inspect = ["src", "api", "webapp", "models", "reports", "scripts", "notebooks"]
    for folder in folders_to_inspect:
        folder_path_local = os.path.join(os.getcwd(), folder)
        if os.path.exists(folder_path_local) and os.path.isdir(folder_path_local):
            print(f"\nContenu de ./{folder}/ :")
            # Utiliser find pour lister tous les fichiers, y compris dans les sous-dossiers, et montrer les .gitkeep
            # os.system(f"find {folder} -print -type d -empty -delete") # Pourrait supprimer les dossiers vides non désirés
            os.system(f"ls -A {folder_path_local}") # Simple ls -A du contenu du dossier
        elif os.path.exists(folder_path_local):
             print(f"\nATTENTION: ./{folder} existe mais n'est pas un dossier.")
        # else:
        #     print(f"\nLe dossier ./{folder}/ n'existe pas localement après la synchronisation.")


    print("\n### 5. OPÉRATIONS GIT (ADD, COMMIT, PUSH) ###")
    print("\n--- Statut Git (après synchronisation et pull) ---")
    !git status

    print("\nAjout de tous les changements au staging area (git add .)...")
    !git add .

    print("\n--- Statut Git (après 'git add .') ---")
    !git status --short # Version courte pour voir les fichiers staged

    commit_message = input("\nEntrez votre message de commit (laissez vide pour annuler le commit/push) : ")

    if not commit_message:
        print("\nMessage de commit vide. Opération de commit et push annulée.")
        print("\n--- Statut Git Actuel (changements non commités) ---")
        !git status
        return

    print(f"\nCommit des changements avec le message : '{commit_message}'")
    os.environ['GIT_COMMIT_MESSAGE'] = commit_message
    !git commit -m "$GIT_COMMIT_MESSAGE"
    del os.environ['GIT_COMMIT_MESSAGE']

    print(f"\nPush des changements vers la branche '{GIT_BRANCH_NAME}' sur GitHub...")
    push_result = os.system(f"git push origin {GIT_BRANCH_NAME}")
    if push_result == 0:
        print("\n--- Opération de synchronisation, commit et push terminée avec succès. ---")
    else:
        print(f"ERREUR : 'git push' a échoué avec le code {push_result}. Vérifiez les messages ci-dessus.")

    print("Vérifiez votre dépôt sur GitHub pour confirmer les changements.")

# --- Exécution de la fonction ---
sync_and_push_to_github_v3()

### 1. VÉRIFICATION DES CHEMINS ###
Source du projet sur Drive : /content/drive/MyDrive/Colab Notebooks/OC/P8
Dépôt Git local dans Colab : /content/OC-autonomous-vehicle-segmentation

### 2. NAVIGATION VERS LE RÉPERTOIRE DU DÉPÔT GIT LOCAL ###
Changé de répertoire vers : /content/OC-autonomous-vehicle-segmentation

### 3. MISE À JOUR DEPUIS LE DÉPÔT DISTANT (GIT PULL) ###
Tentative de 'git pull origin main' --rebase...
AVERTISSEMENT : 'git pull --rebase' a rencontré un problème (code 32768). Vérifiez les messages.
Si des conflits sont signalés, ils devront être résolus manuellement avant de continuer.

### 4. SYNCHRONISATION DES FICHIERS (DRIVE -> REPO LOCAL) ###
   Suppression du dossier de destination existant : /content/OC-autonomous-vehicle-segmentation/notebooks
   Copie du dossier : '/content/drive/MyDrive/Colab Notebooks/OC/P8/notebooks' vers '/content/OC-autonomous-vehicle-segmentation/notebooks'
   Suppression du dossier de destination existant : /content/OC-autonomous-vehicle

In [20]:
%cd /content/OC-autonomous-vehicle-segmentation
!rm Notebook_modélisation.ipynb
!ls # Pour vérifier qu'il n'est plus là à la racine

/content/OC-autonomous-vehicle-segmentation
rm: cannot remove 'Notebook_modélisation.ipynb': No such file or directory
api	Notebook_modélisation.ipynb  README.md	scripts  webapp
models	notebooks		     reports	src


In [22]:
%cd /content/OC-autonomous-vehicle-segmentation
!rm "Notebook_modélisation.ipynb"
!ls # VÉRIFIEZ que ce fichier n'apparaît PLUS dans la liste.

/content/OC-autonomous-vehicle-segmentation
api  models  notebooks	README.md  reports  scripts  src  webapp
